In [ ]:
"""
Package Importation

Import statements to have the necessary packages for data cleaning (np,pd), data structuring (np,pd),
distance calculation (geopy), and datetime calculation (datetime).
"""

# Import pandas with alias 'pd' for easy reference
# Data importation, strucure, cleaning, and some analysis
import pandas as pd

# Import numpy with alias 'np' for easy reference
# Math functions and general numerical calculations
import numpy as np

# Import the distance function from geopy package
# Calculate the distance between 2 latitude/longitude points
import geopy.distance

# Import the datetime function from datetime package
# Convert strings to datetime variables that are able to calculate change in time, pull the day, month, year, etc, 
from datetime import datetime

from dateutil.relativedelta import relativedelta

import fiona

import geopandas as gpd

from shapely.geometry import Point, Polygon

import shapely.wkt

import math

import ast

"""
Data Importation

Import the data download from Metabase query: Postcodes Enriched (All Postcodes v4) as a csv file.

Cumulative added onto from Feb 2022.
"""

# Import input data from a .csv file using the pd.read_csv() function with the name of the file.
# Note: One does not need to specify the full path of the file as long at the .ipynb file is in the same folder as the data
# Read the table from Metabase query Postcodes Enriched (All Postcodes v4)
post = pd.read_csv('postcodes_enriched.csv')
post = post[['postcode']]

In [ ]:
"""
Data Cleaning

Remove any rows with a single missing value and filter out any blank values just in case.
"""
# Drop missing values from the dataset using the dropna() function
post = post.dropna()
# Remove any rows of data where the postcode is an empty string
post = post[post['postcode']!=""]

In [ ]:
"""
Data Cleaning

Due to some issues where a postcode can be a unit # or not be of length 6 (Singapore standard length)
we will have to either ignore those or remove the first character of it.

There are postcodes that start with 'S', however the dataset provided by SingPost does not account for 
that so it has to be stripped out.
"""
# Create an empty list to add the newly cleaned/checked postcodes
new_post = []

# Loop to go through every postcode the Metabase query
for i in post['postcode']:
    
    temp = str(i)
    
    # In the case the postcode starts with a #, remove the postcode entirely
    if i.startswith('#'):
        new_post.append(np.nan)
    
    elif i.startswith('(S)'):
        new_post.append(temp.strip()[3:])
        
    # In the case the postcode starts with a (, remove the first character of the string/text
    elif i.startswith('('):
        new_post.append(temp.strip()[1:])
        
    # In the case the postcode starts with a S, remove the first character of the string/text
    elif i.startswith('S'):
        new_post.append(temp.strip()[1:])
        
    # In the case the whole string consists of 0's, remove the postcode entirely
    elif all(c in '0' for c in i): # All 0
        new_post.append(np.nan)
        
    # In the case where the length of the string is 5 after removing whitespace, when the Singapore standard postcode length is 6
    # This usually applies for postcodes with a leading 0 which will be omitted from the data
    # Remove the white space around the sting and add a leading 0 to it
    elif len(i.strip()) == 5:
        new_post.append("0"+temp.strip())
    
    # In the case where the length of the string is 6 after removing whitespace, this matches the Singapore standard postcode length of 6
    # Remove the white space around the sting
    elif len(i.strip()) == 6:
        new_post.append(temp.strip())
        
    # In the case where the length of the string is 7 after removing whitespace
    # This usually applies for postcodes with some leading character
    # Remove the whitespace around the string and remove the leading character
    elif len(i.strip()) == 7:
        new_post.append(temp.strip()[1:])
    
    # In the case where the length of the string is greater than 7 after removing whitespace
    # Remove the postcode entirely
    elif len(i.strip()) > 7:
        new_post.append(np.nan)
    
    # In any other case, remove the postcode entirely
    # To maintain a level of cleanliness and ensure the length of the newly created list matches the length of postv2
    else:
        new_post.append(np.nan)
        
"""
Data Cleaning

Append the cleaned postcodes to the data frame (df) and remove any that are missing.

Reset the df's index so when we iterate through it with iloc it will align properly.
"""

# Create a new column to hold the cleaned postcodes
post['postcode'] = new_post

# Remove any of the new cleaned postcodes that are NA
post = post.dropna(subset=['postcode'])

post = post.sort_values(by='postcode')

# Reset the index and remove the created index column to keep the dataset clean
post = post.reset_index().drop(columns='index',axis=1)

post = post.drop_duplicates()

In [ ]:
post[post['postcode'].str.len()==6]

In [ ]:
"""
Import postcode data with respective lat/long values
"""

sg_addresses = pd.read_csv('sg_addresses.csv',low_memory=False)
sg_addresses = sg_addresses[['postcode','latitude','longitude']].sort_values('postcode')
sg_addresses = sg_addresses.drop_duplicates(subset = ['postcode']).reset_index().drop(columns='index',axis=1)

post2 = post.merge(sg_addresses[['postcode','latitude','longitude']], on = 'postcode', how = 'left')

In [ ]:
# sg_addresses[sg_addresses['postcode'].str.startswith('0')] #Both datasets can start with 0
# post[post['postcode'].str.startswith('0')]

In [ ]:
post2 = post2.dropna(subset=['latitude'])
post2

In [ ]:
fiona.drvsupport.supported_drivers['kml'] = 'rw'  # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['KML'] = 'rw'  # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers ['LIBKML'] = 'rw'  # enable KML support which is disabled by default

In [ ]:
"""
Combine lat/long into 1 column with data type POINT for route zone allocation
"""
postcodes_points = []
for xy in zip(post2['longitude'], post2['latitude']):
    postcodes_points.append(Point(xy))

In [ ]:
geo_postcodes = gpd.GeoDataFrame(post2[['postcode','longitude','latitude']],
                                 crs = {'init': 'epsg:4326'},
                                 geometry = postcodes_points)
geo_postcodes = geo_postcodes.reset_index().drop(columns='index',axis=1)
geo_postcodes

In [ ]:
# Issue with how polygons are store, most are stored as 'Polygon', but some are stored as 'MultiPolygon' - lat/longs are stored as a 4D
sg = pd.read_csv('sg_zones.csv')
sg2 = sg[['hub_id','name','short_name','polygon']]

geo = []
route_zone = []

crs = 'epsg:4326'
for i in range(0, len(sg2['polygon'])):
    temp = sg2.iloc[i]['polygon']
    temp2 = ast.literal_eval(temp)
    temp3 = temp2['geometry']['coordinates']
    
    if all(c in '[' for c in str(temp3)[0:4]): # 
        for j in temp3:
            temp4 = [x for td in j for x in td]
            long = [i[0] for i in temp4]
            lat = [i[1] for i in temp4]
            try:   
                poly = Polygon(zip(long, lat))
            except:
                continue
            geo.append(poly)
            route_zone.append(sg2.iloc[i]['short_name'])
            
    else:
        temp4 = [x for td in temp3 for x in td]
        long = [i[0] for i in temp4]
        lat = [i[1] for i in temp4]
        try:   
            poly = Polygon(zip(long, lat))
        except:
            continue
        geo.append(poly)
        route_zone.append(sg2.iloc[i]['short_name'])


sg_zones = gpd.GeoDataFrame(geometry=geo, crs=crs)
sg_zones['route_zone']= route_zone
sg_zones

In [ ]:
# Consider including conversion script to take route zones from addressing_prod_gl.zones to polygons/multipolygons
# sg_zones = pd.read_csv('route_zones.csv')

# sg_zones = sg_zones.drop(axis=1, columns=['Unnamed: 0'])
# sg_zones['geometry'] = sg_zones['geometry'].apply(lambda x:shapely.wkt.loads(x))
# sg_zones

# sg_zones = gpd.GeoDataFrame(sg_zones[['route_zone']],
#                             crs = {'init': 'epsg:4326'},
#                             geometry = sg_zones['geometry'])
# sg_zones

In [ ]:
"""
Allocate each postcode their respective route zone
"""

import time
start = time.time()

postcode = []
route_zone = []
lat = []
long = []

for i in range(0, len(geo_postcodes['geometry'])):
    postcode_geo = geo_postcodes.iloc[i]['geometry']
    
    for j in range(0,len(sg_zones['route_zone'])):
        zone_polygon = sg_zones.iloc[j]['geometry']
        
        if zone_polygon.contains(postcode_geo):
            postcode.append(geo_postcodes.iloc[i]['postcode'])
            route_zone.append(sg_zones.iloc[j]['route_zone'])
            lat.append(geo_postcodes.iloc[i]['latitude'])
            long.append(geo_postcodes.iloc[i]['longitude'])
            break
    else:
        postcode.append(geo_postcodes.iloc[i]['postcode'])
        route_zone.append(np.nan)
        lat.append(geo_postcodes.iloc[i]['latitude'])
        long.append(geo_postcodes.iloc[i]['longitude'])

end = time.time()
print( (end - start)/60 )

In [ ]:
post3 = pd.DataFrame({'postcode':postcode, 'post_lat': lat, 'post_long': long, 'route_zone': route_zone})
post3

In [ ]:
post3[post3['route_zone'].str.startswith('MAINCON',na=False)]

In [ ]:
post3[post3.postcode=='600311']

In [ ]:
"""
Repeat route zone allocation for FM zones
"""

fm_zones = gpd.read_file('FM.kml', driver='LIBKML')
fm_zones2 = fm_zones[['Name','geometry']]

In [ ]:
fm_zones2

In [ ]:
fm_zone = []
fm_zone_name = []
for i in range(0,len(fm_zones2)):
    fm_zone.append(fm_zones2.iloc[i]['Name'])
    fm_zone_name.append(fm_zones2.iloc[i]['Name'].split('-')[0].split(' ',1)[1].strip())

In [ ]:
"""
Find any FM zones that does not start with East/North/Central/West
"""

fm_weird_names = pd.DataFrame({'fm_zone':fm_zone, 'fm_zone_name':fm_zone_name })
fm_weird_names2 = fm_weird_names[ (~fm_weird_names['fm_zone_name'].str.startswith('East')) 
               & (~fm_weird_names['fm_zone_name'].str.startswith('North')) 
               & (~fm_weird_names['fm_zone_name'].str.startswith('Central'))
               & (~fm_weird_names['fm_zone_name'].str.startswith('West')) ]
fm_weird_names2.to_csv('fm_weird_names.csv')

In [ ]:
start = time.time()

fm_group = []
fm_team = []

for i in range(0, len(geo_postcodes['geometry'])):
    postcode_geo = geo_postcodes.iloc[i]['geometry']
    
    for j in range(0,len(fm_zones2['Name'])):
        zone_polygon = fm_zones2.iloc[j]['geometry']
        
        if zone_polygon.contains(postcode_geo):
            team = fm_zones2.iloc[j]['Name'].replace('\t','').replace(u'\xa0', u' ').split('-')[0].split(' ',1)[1].strip()
            fm_team.append(team)
            fm_group.append(team.split(' ')[0].strip())
            break
    else:
        fm_team.append(np.nan)
        fm_group.append(np.nan)

end = time.time()
print( (end - start)/60 )

In [ ]:
post3['fm_team'] = fm_team
post3['fm_group'] = fm_group

In [ ]:
post3['fm_team'] = post3['fm_team'].str.upper()
post3['fm_group'] = post3['fm_group'].str.upper()

In [ ]:
# post3[post3['fm_team'].str.startswith('Staff',na=False)]

In [ ]:
"""
Data Importation

Import data from the previous iteration of Postcodes Enriched: postcodes_enriched_table_old.csv

This dataset has the previous mappings of each postcode and its respective route zone
"""

# Read the table from previous iteration of Postcodes Enriched v5
old_route_zones = pd.read_csv('postcodes_enriched_table_old.csv')

# Extract the key variables of interest: previous postcode (key) and route zone
prev_route_zones = old_route_zones[['postcode','route_zone']]

prev_route_zones['postcode'] = prev_route_zones['postcode'].astype(str)

prev_route_zones
# Creating a necessary lists to hold information and build the legacy route zone
new_post2 = []

# Loop to go through every postcode of the previous iteration of postcodes enriched
for i in prev_route_zones['postcode']:
    
    # Same cleaning step as before
    # Will need to be modified if changes are made above
    temp = str(i)
    if temp.startswith('#'):
        new_post2.append(np.nan)
    elif temp.startswith('('):
        new_post2.append(temp.strip()[1:])
    elif temp.startswith('S'):
        new_post2.append(temp.strip()[1:])
    elif all(c in '0' for c in temp): # All 0
        new_post2.append(np.nan)
    elif len(temp) == 5:
        new_post2.append("0"+temp.strip())
    elif len(temp) == 6:
        new_post2.append(temp.strip())
    elif len(temp) == 7:
        new_post2.append(temp.strip()[1:])
    elif len(temp) > 7:
        new_post2.append(np.nan)
    else:
        new_post2.append(np.nan)

# Create new variables to hold the cleaned legacy postcodes
prev_route_zones['postcode'] = new_post2

# # Remove the previous 'uncleaned' postcodes and rename the newly cleaned postcodes 
prev_route_zones = prev_route_zones.rename(columns={"route_zone":"prev_route_zone"})

# # Merge/Left join the data together, adding the previous route zone
post4 = post3.merge(prev_route_zones,on='postcode')

In [ ]:
post4['route_zone'] = post4['route_zone'].astype(str)

In [ ]:
post4[post4.postcode=='600311']

In [ ]:
post4 = post4.drop_duplicates(subset=['postcode'])

In [ ]:
# Creating a necessary lists to hold information and build the current route zone
subzone = []

# Loop to go through every current route zone
for i in post4['route_zone']:
    
    if i == 'nan':
        subzone.append(np.nan)
        continue
        
    # Extract the subzone from each route zone by taking the evrything before the first -
    subzone.append(i.split('-')[0].strip())
    

# Create new variables to hold the current subzone
post4['subzone'] = subzone

In [ ]:
####

"""
Data Importation

Import the data download from Metabase: Postcodes Enriched (Hub's Route Zones v4).

This dataset has the mappings of each route_zone to its proper hub.

If a hub gets 'turned off', route_zones under it should be updated and redirected to another hub.
"""
station_route_zones = pd.read_csv('station_route_zones.csv')
# station_route_zones = pd.read_csv('station_route_zones_temp.csv')

# Remove duplicate route zones just in case as 1 route zone can only belong to 1 station/hub
station_route_zones = station_route_zones.drop_duplicates('route_zone')

"""
Data Joining

Join the 2 previous datasets to postcodes.
"""
# Merge/Left join the data together, adding the proper destination station to each route zone
post5 = post4.merge(station_route_zones, on='route_zone',how='left')

# Rename the columns to something more descriptive
post5 = post5.rename(columns={'station_name':'dest_station', 'station_id':'dest_station_id'})

# Drop additional columns
# post5 = post5.drop(columns=['name','route_zone_id'])
post5 = post5.drop(columns=['route_zone_id'])

In [ ]:
post5[post5.postcode=='600311']

In [ ]:

"""
Data Creation

Calculate the distance between each postal code to its respective station.
"""

# Creating a necessary lists to hold information and build the distance from the centroid of a route zone to its destination station
dist_station_post = []


# Loop to go through every postcode's route zone centroid
for i in post5['postcode']:
    
    temp = post5[post5.postcode == i]
#     print(temp['post_lat'].values[0])
    
    # Check if any of the key variables in the calculation is null/NaN/missing, if any are missing then skip over this postcode
    if temp.isnull().values.any():
        dist_station_post.append(np.nan)
        continue
    else:
    # Save each variable separately
        post_lat = temp['post_lat'].values[0]
        post_long = temp['post_long'].values[0]
        station_lat = temp['station_lat'].values[0]
        station_long = temp['station_long'].values[0]
    
    # Calculate the distance from the centroid of a route zone to its destination station
    dist_station_post.append(geopy.distance.distance( (post_lat,post_long),(station_lat,station_long) ).km)

# Create new variables to hold the distance calculated above
post5['dist_from_station_to_post'] = dist_station_post

# post5

In [ ]:
post5[post5.postcode=='600311']

In [ ]:
"""
Data Importation

Import the data download from Metabase: Postcodes Enriched(Parcel volume of postcode v4)

This dataset has the both doorstep, PUDO, and RPU orders for all postcodes within the previous month.

Import the data download from Redash: calendar.csv

19th of the previous month to the 19th of the current month.
"""
postcode_volume = pd.read_csv('postcodes_volume.csv')
postcode_volume = postcode_volume.dropna()
calendar = pd.read_csv('calendar.csv') # Update for next year

new_post3 = []

# Loop through every postcode with 'recent' (19th to 19th) volume
for i in postcode_volume['postcode']:
    
    # Same cleaning step as before
    # Will need to be modified if changes are made above
    temp = str(i)
    if temp.startswith('#'):
        new_post3.append(np.nan)
    elif temp.startswith('S'):
        new_post3.append(temp.strip()[1:])
    elif temp.startswith('('):
        new_post3.append(temp.strip()[1:])
    elif all(c in '0' for c in temp): # All 0
        new_post3.append(np.nan)
    elif len(temp) == 5:
        new_post3.append("0"+temp.strip())
    elif len(temp) == 6:
        new_post3.append(temp.strip())
    elif len(temp) == 7:
        new_post3.append(temp.strip()[1:])
    elif len(temp) > 7:
        new_post3.append(np.nan)
    else:
        new_post3.append(np.nan)

"""
Data Creation

Calculate the volume for each postcode and the average volume (total/working_days) using volume and no of working days.
"""

# Create a new variable to hold the 'monthly' volume
postcode_volume['postcode']=new_post3
postcode_volume2 = postcode_volume.groupby('postcode').sum()

# Merge/Left join the data together, adding monthly volume for each postcode
post6 = post5.merge(postcode_volume2,on='postcode',how='left')

# Rename certain variables to be more descriptive
post6 = post6.rename(columns={'volume':'monthly_volume'})

# Get the current date's month
current_date = datetime.today().strftime('%Y-%m')
date_before_month = datetime.today() - relativedelta(months=1)


# Calculate the number of working days between the 19th to 19th
no_working_days = calendar[calendar['date']==current_date+'-19']['working_day_cum'].values[0] - calendar[calendar['date']== date_before_month.strftime('%Y-%m')+'-19']['working_day_cum'].values[0]

# Calculate the average daily volume: monthly volume / no working days and create a new variable for it
post6['avg_daily_volume'] = round(post6['monthly_volume']/no_working_days,2)

In [ ]:
post6[post6.postcode=='134028']

In [ ]:
"""
Data Importation

Import the data download from Fleet: FM.csv

This dataset shows the mapping of each subzone to WEST, CENTRAL, NORTH, EAST First Mile (FM) groups, teams and 
the proper dropoff hub (CK).
"""

"""
Data Importation

Import the data download from Fleet: hubs.csv

This dataset shows the lat and long of hubs PANDAN and YCK.
"""

main_hubs = pd.read_csv('hubs.csv')

post7 = post6

post7 = post7.drop_duplicates(subset='postcode')

post7['fm_dropoff_hub'] = 'YCK'

# Merge/Left join the data together, adding dropoff hub information (lat/long)
post8 = post7.merge(main_hubs,on='fm_dropoff_hub',how='left')

# Remove unused data
post8 = post8.drop(columns=['hub_id'])

post8['fm_group'] = post8['fm_group'].str.upper()
post8['fm_team'] = post8['fm_team'].str.upper()
post8

In [ ]:
# 69 postocdes could not be mapped, J08-F, K06-G, K06-E, G01-G, F10-D
# post8[post8.fm_team.isnull()]

In [ ]:
post8[post8.postcode=='600311']

In [ ]:
temp = post8

count = 0

for i in range(0, len(temp['fm_group'])):
    temp2 = temp.iloc[0]
    group = temp2['fm_group']
    team = temp2['fm_team'].split(' ')[0]
    
    if group != team:
        count += 1
count

In [ ]:
# Standardize names and add a checker here to inform Fel if any teams/groups are different

In [ ]:
"""
Data Creation

Calculate the distance between each postcode to their respective FM dropoff hub (PANDAN or YCK).
"""

# Creating a necessary lists to hold information and build distance from each postcode to dropoff hub
dist_dropoff_hub = []

# Loop through every postcode's dropoff hub
for i in range(0,len(post8['fm_dropoff_hub_latitude'])):
    temp = post8.iloc[i]
    
    # Check if any of the key variables in the calculation is null/NaN/missing, if any are missing then skip over this postcode
    if np.isnan(temp['post_lat']) or np.isnan(temp['post_long']) or np.isnan(temp['fm_dropoff_hub_latitude']) or np.isnan(temp['fm_dropoff_hub_longitude']):
        dist_dropoff_hub.append(np.nan)
        continue
    else:
        # Save each variable separately
        post_lat = temp['post_lat']
        post_long = temp['post_long']
        main_hub_lat = temp['fm_dropoff_hub_latitude']
        main_hub_long = temp['fm_dropoff_hub_longitude']
    
    # Calculate the distance from the centroid of a route zone to its destination station
    dist_dropoff_hub.append(geopy.distance.distance( (post_lat,post_long),(main_hub_lat,main_hub_long) ).km)

# Create new variables to hold the distance from each postcode to dropoff hub
post8['dist_from_post_to_dropoff_hub'] = dist_dropoff_hub

post8 = post8.drop(columns=['fm_dropoff_hub_latitude','fm_dropoff_hub_longitude'])

post8

In [ ]:
post8[post8.postcode=='600311']

In [ ]:
"""
Data Importation

Import the data download from OPEX: restricted.csv

This dataset shows all the postcodes we 'should not' be able to service. But there are some we do service anyway.
"""
# Will have to check in with OPEX every quarter or so to check if this list has changed
restricted = pd.read_csv('restricted.csv')
new_restricted_post = []

"""
Data Cleaning

Code here is a bit too much as the main issue is that postcodes that start with '0' loses the '0' so it adds it back.
"""

# Loop through every restricted postcode
for i in restricted['Postal Code']:
    
    # Same cleaning step as before
    # Will need to be modified if changes are made above
    temp = str(i)
    if temp.startswith('#'):
        new_restricted_post.append(np.nan)
    elif temp.startswith('S'):
        new_restricted_post.append(temp[1:])
    elif temp.startswith('('):
        new_restricted_post.append(temp[1:])
    elif all(c in '0' for c in temp): # All 0
        new_restricted_post.append(np.nan)
    elif len(temp) == 5:
        new_restricted_post.append("0"+temp)
    elif len(temp) == 6:
        new_restricted_post.append(temp)
    elif len(temp) == 7:
        new_restricted_post.append(temp[1:])
    elif len(temp) > 7:
        new_restricted_post.append(np.nan)
    else:
        new_restricted_post.append(np.nan)

"""
Data Creation

If the postal code is restricted then 1 else 0. This will act as a flag to indicate which postcodes are restricted.
"""
# Creating a necessary lists to hold information and build which postcodes are restricted
restricted_flag = []

# Loop through every postcode in the base dataset
for i in post8['postcode']:
    
    # In the case the postcode matches one in the restricted list then flag as 1
    if i in np.array(new_restricted_post):
        restricted_flag.append(1)
    # In the case it is not, flag as 0
    else:
        restricted_flag.append(0)

# Create new variables to hold the restricted flag
post8['restricted_zone_flag'] = restricted_flag
post8

In [ ]:
# post8[post8.postcode=='018895']
post8[post8.postcode=='600311']

In [ ]:
"""
Data Importation

Import the data download from SingPost: 6D.csv

This dataset has the full list of postcodes in Singapore and their building_type.
"""
# This is the base dataset from SingPost that we will add to as it is a 'more' encompassing dataset
post6D = pd.read_csv('6D.csv')

# Creating a necessary lists to hold information and build the base postcodes dataset
new_post4 = []

"""
Data Cleaning

Code here is a bit too much as the main issue is that postcodes that start with '0' loses the '0' so it adds it back.
"""      
# Loop to go through every 6D postcode
for i in post6D['POSTCODE']:
    
    # Same cleaning step as before
    # Will need to be modified if changes are made above
    temp = str(i)
    if temp.startswith('#'):
        new_post4.append(np.nan)
    elif temp.startswith('S'):
        new_post4.append(temp.strip()[1:])
    elif temp.startswith('('):
        new_post4.append(temp.strip()[1:])
    elif all(c in '0' for c in temp): # All 0
        new_post4.append(np.nan)
    elif len(temp) == 5:
        new_post4.append("0"+temp.strip())
    elif len(temp) == 6:
        new_post4.append(temp.strip())
    elif len(temp) == 7:
        new_post4.append(temp.strip()[1:])
    elif len(temp) > 7:
        new_post4.append(np.nan)
    else:
        new_post4.append(np.nan)

# Create new variables to hold the cleaned 6D postcodes
post6D['POSTCODE'] = new_post4

"""
Data Relabelling

To simplify the building codes from 8 categories and 66 codes into 1 category and 15 codes.

Combine similiar building types like Condo and Condo with Retail.
"""
# Creating a necessary lists to hold information and build the postcode building type
new_bldgcode = []

# Extract the building types from the 6D dataset
bldgcode = post6D['BLDGCODE']

# Loop to go through every building type designation
for i in bldgcode:
    
    # Relabelling SingPost's building type designations to our own X# code
    # For a more comprehensive breakdown, refer below and to ...
    if str(i) == 'C03' or str(i) == 'C11' or str(i) == 'T01' or str(i) == 'T02':
        new_bldgcode.append("X01") #C03
        
    elif str(i) == 'C04' or str(i) == 'C01' or str(i) == 'C02' or str(i) == 'C14':
        new_bldgcode.append("X02") #C04
        
    elif str(i) == 'C06' or str(i) == 'C05' or str(i) == 'C07':
        new_bldgcode.append("X03") #C06
        
    elif str(i) == 'C08':
        new_bldgcode.append("X04")
        
    elif str(i) == 'C09' or str(i) == 'C10':
        new_bldgcode.append("X05")
        
    elif str(i) == 'C12' or str(i) == 'C13' or str(i) == 'T03' or str(i) == 'T04':
        new_bldgcode.append("X06")
    
    elif str(i) == 'R01':
        new_bldgcode.append("X07")
    
    elif str(i) == 'R02' or str(i) == 'R03' or str(i) == 'R06':
        new_bldgcode.append("X08")
    
    elif str(i) == 'R04' or str(i) == 'R05':
        new_bldgcode.append("X09")
        
    elif str(i) == 'E01' or str(i) == 'E02' or str(i) == 'E03' or str(i) == 'E06':
        new_bldgcode.append("X10")
    
    elif str(i) == 'E04' or str(i) == 'E05' or str(i) == 'E07':
        new_bldgcode.append("X11")
        
    elif str(i).startswith("L"):
        new_bldgcode.append("X12")
    
    elif str(i).startswith("H"):
        new_bldgcode.append("X13")
    
    elif str(i).startswith("W"):
        new_bldgcode.append("X14")
        
    elif str(i).startswith("Z"):
        new_bldgcode.append("X15")
    
    else:
        new_bldgcode.append(np.nan)
        
# Create new variables to hold the relabelled building type codes
post6D['new_bldgcode'] = new_bldgcode

# Drop duplicate cleaned postcodes (Very unlikely, but just in case)
post6D = post6D.drop_duplicates('POSTCODE')
post6D

In [ ]:
"""
Data Creation

With the new building_code established (X01 - X15), label each with a building_type.
"""
# Creating a necessary lists to hold information and build the building type name
bldg_type = []

# Loop through every building type code of each postcode
for i in post6D['new_bldgcode']:
    
    # Label each building type code with its proper name
    if i == "X01":
        bldg_type.append("Shophouse/Walkup")
    elif i == "X02":
        bldg_type.append("Industrial/Factory/Warehouse")
    elif i == "X03":
        bldg_type.append("Office")
    elif i == "X04":
        bldg_type.append("Shopping/Dining")
    elif i == "X05":
        bldg_type.append("Hotel/Hostel/Works_Dorms")
    elif i == "X06":
        bldg_type.append("Port/Shipyard")
    elif i == "X07":
        bldg_type.append("HDB")
    elif i == "X08":
        bldg_type.append("Condo")
    elif i == "X09":
        bldg_type.append("Landed")
    elif i == "X10":
        bldg_type.append("Educational_Institute")
    elif i == "X11":
        bldg_type.append("Higher_Education")
    elif i == "X12":
        bldg_type.append("Community_Club")
    elif i == "X13":
        bldg_type.append("Medical_Facility")
    elif i == "X14":
        bldg_type.append("Religious_Organization")
    elif i == "X15":
        bldg_type.append("Misc_Buildings")
    else:
        bldg_type.append(np.nan)
        
# Create new variables to hold the building type name
post6D['bldg_type'] = bldg_type

post6D

In [ ]:
"""
Data Joining

Join the 6D dataset to postcodes and standardize the columns names.
"""
# Merge/Left join the data together, using the 6D dataset as a base and joining our previously calculated values to it
post9 = post6D.merge(post8, left_on='POSTCODE', right_on="postcode",how='left')

# Remove old variables
post9 = post9.drop(columns=['postcode', 'BLDGNO', 'STREETNAME', 'BLDGNAME',
                              'BLDGNAME', 'BLDGCODE', 'BLDGDESC'])

# Rename certain variables to be more in tune with other variables
post9 = post9.rename(columns={'new_bldgcode':'bldg_code', 'POSTCODE':'postcode'})

In [ ]:
post9[post9.postcode=='600311']

In [ ]:
"""
Import Postcodes Index
"""

PI = pd.read_csv('PI_full.csv')

In [ ]:
# Creating a necessary lists to hold information and build the base postcodes dataset
new_post5 = []

"""
Data Cleaning

Code here is a bit too much as the main issue is that postcodes that start with '0' loses the '0' so it adds it back.
"""      
# Loop to go through every 6D postcode
for i in PI['postcode']:
    
    # Same cleaning step as before
    # Will need to be modified if changes are made above
    temp = str(i)
    if temp.startswith('#'):
        new_post5.append(np.nan)
    elif temp.startswith('S'):
        new_post5.append(temp.strip()[1:])
    elif temp.startswith('('):
        new_post5.append(temp.strip()[1:])
    elif all(c in '0' for c in temp): # All 0
        new_post5.append(np.nan)
    elif len(temp) == 5:
        new_post5.append("0"+temp.strip())
    elif len(temp) == 6:
        new_post5.append(temp.strip())
    elif len(temp) == 7:
        new_post5.append(temp.strip()[1:])
    elif len(temp) > 7:
        new_post5.append(np.nan)
    else:
        new_post5.append(np.nan)

# Create new variables to hold the 
PI['postcode'] = new_post5
PI

In [ ]:
post10 = post9.merge(PI[['postcode','pi_within','pi_across_short','pi_across_long']], on='postcode', how='left')
post10 = post10.rename(columns={'pi_within':'difficulty_within','pi_across_short':'difficulty_across_short','pi_across_long':'difficulty_across_long'})

In [ ]:
# post10

In [ ]:
"""
Data Creation

Created and expiry dates are used to keep track of which version of the table we are looking at.

When a 6 month rolling window (6 months of data) is keep, we can use the 2 dates to backtrack.
"""

current_date = datetime.today().strftime('%m/%y')

date_after_month = datetime.today()+ relativedelta(months=1)

# Timestamp of when the table was roughly created and roughly when it will be outdated
# 19th of the current month
post10['created_on'] = '19/'+ current_date


# 19th of the following month
# At the end of the every year, must switch to the following year
post10['expiry_on'] = '19/'+ date_after_month.strftime('%m/%y')

post10

In [ ]:
"""
Import districts of SG data and match the first 2 digits of each postcode to a district
"""

districts = pd.read_csv('districts.csv')
new_post6 = []
for i in districts['postcode sector']:
    if len( str(i) ) == 1:
        new_post6.append('0' + str(i))
    else:
        new_post6.append(str(i))
districts['leading_post'] = new_post6

post10['temp_postcode'] = post10['postcode'].str[0:2]

post11 = post10.merge(districts[['leading_post','state']], left_on='temp_postcode', right_on='leading_post', how='left')
post11 = post11.drop(['leading_post','temp_postcode'],axis=1)
post11

In [ ]:
post12 = post11[['postcode','post_lat', 'post_long','bldg_type','route_zone',
                 'subzone','prev_route_zone','dest_station_id','dest_station', 'station_lat', 'station_long',
                 'dist_from_station_to_post','monthly_volume','avg_daily_volume',
                 'fm_group','fm_team','fm_dropoff_hub','dist_from_post_to_dropoff_hub','restricted_zone_flag',
                 'difficulty_within','difficulty_across_short','difficulty_across_long','state','created_on','expiry_on']]

In [ ]:
"""
Data Export
"""

# Add a month year timestamp for easy distinction of iterations of Postcode Enriched
timestamp = datetime.today().strftime("%B_%Y")

# Table export
# post12.to_csv('postcodes_enriched_table_v5.5_'+timestamp+'.csv')
post12.to_csv('postcodes_enriched_table_v5.5_'+timestamp+'.csv',index=False)

In [ ]:
import pandas as pd
post12 = pd.read_csv('postcodes_enriched_table_v5.5_'+timestamp+'.csv')

In [ ]:
"""
Adjust postcodes difficulty data for upload to Metabase
"""

post13 = post12.copy()
post13['difficulty_across'] = (post13['difficulty_across_short'] + post13['difficulty_across_long'])/2
post13 = post13.drop(axis=1,columns=['difficulty_across_short','difficulty_across_long'])

post14 = post13[['postcode','post_lat', 'post_long','bldg_type','route_zone',
                 'subzone','prev_route_zone','dest_station_id','dest_station', 'station_lat', 'station_long',
                 'dist_from_station_to_post','monthly_volume','avg_daily_volume',
                 'fm_group','fm_team','fm_dropoff_hub','dist_from_post_to_dropoff_hub','restricted_zone_flag',
                 'difficulty_within','difficulty_across','state','created_on','expiry_on']]
post14.to_csv('postcodes_enriched_table_v5.5_'+timestamp+'_to_upload.csv',index=False)

In [ ]:
post12[post12.postcode==134028]['monthly_volume']

In [ ]:
# post12[post12.route_zone.str.contains('(25/2)',na=False)]

In [ ]:
bldg = post12.groupby('bldg_type')['monthly_volume'].sum()
bldg2 = bldg.reset_index()
bldg2.to_csv('bldg.csv',index=False)

In [ ]:
station = post12.groupby('dest_station')['monthly_volume'].sum()
station2 = station.reset_index()
station2.to_csv('station.csv',index=False)

In [ ]:
route_zone = post12.groupby('route_zone')['monthly_volume'].sum()
route_zone2 = route_zone.reset_index()
route_zone2.to_csv('route_zone.csv',index=False)

In [ ]:
route_zone_bldg = post12.groupby(['route_zone','bldg_type'])['monthly_volume'].sum()
route_zone_bldg2 = route_zone_bldg.reset_index()
route_zone_bldg2.to_csv('route_zone_bldg.csv',index=False)

In [ ]:
post12.head()